# Functions being used to extract features:

In [2]:
from types import new_class
import urllib.request
import requests
from bs4 import BeautifulSoup

def get_title(new_soup):
  try:
    title = new_soup.find("div", attrs={"class": "well"}).find("h1").text.strip()
    return title
  except AttributeError:
    return None

def get_model(new_soup):
  try:
    model = new_soup.find("td").find("p").text.strip()
    return model
  except AttributeError:
    return None

def get_location(new_soup):
  try:
    location = new_soup.find("div", attrs={"class": "well"}).find("p", attrs={"class":"detail-sub-heading"}).find("a").text.strip()
    return location
  except AttributeError:
    return None

def get_mileage(new_soup):
    try:
        all_td = new_soup.find_all("td")
        mileage = all_td[1].find("p").text.strip()
        return mileage
        print("Mileage not found.")
        return None

    except AttributeError:
        return None

def get_price(new_soup):
  try:
    price = new_soup.find("div", attrs={"class": "price-box"}).find("strong").text.strip()
    return price
  except AttributeError:
    return None

def get_color(new_soup):
  try:
      ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})

      if ul:
          items = ul.find_all("li")
          for i in range(len(items)):
              if "Color" in items[i].text:
                 color = items[i + 1].text.strip()
                 return color
      else:
          print("UL not found.")
  except AttributeError:
    return None

def get_bodytype(new_soup):
  try:
    ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})

    if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Body Type" in items[i].text:
                bodytype = items[i + 1].text.strip()
                return bodytype
  except AttributeError:
    return None

def get_assembly(new_soup):
  try:
    ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})
    if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Assembly" in items[i].text:
                assembly = items[i + 1].text.strip()
                return assembly

  except AttributeError:
    return None

def get_registeredin(new_soup):
  try:
    ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})
    if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Registered In" in items[i].text:
                registeredin = items[i + 1].text.strip()
                return registeredin
  except AttributeError:
    return None

def get_enginecapacity(new_soup):
  try:
     ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})
     if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Engine Capacity" in items[i].text:
                enginecapacity = items[i + 1].text.strip()
                return enginecapacity
  except AttributeError:
    return None


def get_referencenumber(new_soup):
  try:
     ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})
     if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Ad Ref #" in items[i].text:
                RefNo = items[i + 1].text.strip()
                return RefNo
  except AttributeError:
    return None

def get_lastupdated(new_soup):
  try:
     ul = new_soup.find("ul",attrs= {"class": "list-unstyled ul-featured clearfix"})
     if ul:
        items = ul.find_all("li")
        for i in range(len(items)):
            if "Last Updated:" in items[i].text:
                LastUpdated = items[i + 1].text.strip()
                return LastUpdated
  except AttributeError:
    return None


def get_fueltype(new_soup):
    try:
        all_td = new_soup.find_all("td")
        fuel_type = all_td[2].find("p").text.strip()
        return fuel_type
        print("fueltype not found.")
        return None

    except AttributeError:
        return None

def get_transmission(new_soup):
    try:
        all_td = new_soup.find_all("td")
        transmission = all_td[3].find("p").text.strip()
        return transmission
        print("transmission not found.")
        return None
    except AttributeError:
        return None



# Main Code:

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {'User-Agent': 'Mozilla/5.0'}
url = 'https://www.pakwheels.com/used-cars/search/-/'
linked_list = []


d = {"Title": [], "Mileage": [], "Model": [], "Color": [],"BodyType": [], "EngineCapacity": [], "FuelType": [],"Transmission": [], "RegisteredIn": [], "Assembly": [],"Price": [], "Location": [],"LastUpdated":[],"ReferenceNo":[]}
x = 0
# only scraped first 100 pages
while x < 100:
    print(f"Fetching listings from: {url}")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    links = soup.find_all("a", attrs={'class': 'car-name ad-detail-path'})
    for link in links:
        href = link.get('href')
        if href and href not in linked_list:
            linked_list.append(href)

    next_page = soup.find("li", class_="next_page")
    if next_page and next_page.a:
        url = "https://www.pakwheels.com" + next_page.a['href']
        x = x + 1
    else:
        url = None

base_url = "https://www.pakwheels.com"

for idx, link in enumerate(linked_list):
    full_url = base_url + link
    print(f"[{idx + 1}/{len(linked_list)}] Scraping: {full_url}")

    try:
        response = requests.get(full_url, headers=headers)
        new_soup = BeautifulSoup(response.content, "html.parser")

        d["Title"].append(get_title(new_soup))
        d["Mileage"].append(get_mileage(new_soup))
        d["Model"].append(get_model(new_soup))
        d["Color"].append(get_color(new_soup))
        d["BodyType"].append(get_bodytype(new_soup))
        d["EngineCapacity"].append(get_enginecapacity(new_soup))
        d["FuelType"].append(get_fueltype(new_soup))
        d["Transmission"].append(get_transmission(new_soup))
        d["RegisteredIn"].append(get_registeredin(new_soup))
        d["Assembly"].append(get_assembly(new_soup))
        d["Price"].append(get_price(new_soup))
        d["Location"].append(get_location(new_soup))
        d["LastUpdated"].append(get_lastupdated(new_soup))
        d["ReferenceNo"].append(get_referencenumber(new_soup))

        time.sleep(1)

    except Exception as e:
        print(f"Failed to scrape: {full_url}\nError: {e}")
        continue

# Save to CSV
cars_df = pd.DataFrame.from_dict(d)
cars_df.to_csv('PakistanUsedCarsData.csv', index=False)
print("Scraping complete. Data saved to 'PakistanUsedCarsData.csv'")


Fetching listings from: https://www.pakwheels.com/used-cars/search/-/
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=2
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=3
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=4
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=5
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=6
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=7
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=8
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=9
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=10
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=11
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=12
Fetching listings from: https://www.pakwheels.com/used-cars/search/-/?page=13
Fe